<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Research_Internship(Wear_Particle)_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#tf.keras.utils.set_random_seed(1)
#tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
num_classes = len(class_names)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Conv2D(16, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(512, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 sequential_2 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                  

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
epochs=70
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[model_checkpoint_callback]
)

Epoch 1/70
19/19 [==============================] - 6s 224ms/step - loss: 1.4965 - accuracy: 0.4679 - val_loss: 1.4103 - val_accuracy: 0.4662
Epoch 2/70
19/19 [==============================] - 2s 79ms/step - loss: 1.2160 - accuracy: 0.4916 - val_loss: 1.1775 - val_accuracy: 0.4662
Epoch 3/70
19/19 [==============================] - 4s 202ms/step - loss: 1.0065 - accuracy: 0.4916 - val_loss: 1.0944 - val_accuracy: 0.5608
Epoch 4/70
19/19 [==============================] - 5s 222ms/step - loss: 0.9960 - accuracy: 0.5726 - val_loss: 1.2476 - val_accuracy: 0.5743
Epoch 5/70
19/19 [==============================] - 4s 212ms/step - loss: 0.9348 - accuracy: 0.5693 - val_loss: 0.9415 - val_accuracy: 0.6081
Epoch 6/70
19/19 [==============================] - 2s 81ms/step - loss: 0.9003 - accuracy: 0.5963 - val_loss: 0.9856 - val_accuracy: 0.5608
Epoch 7/70
19/19 [==============================] - 2s 77ms/step - loss: 0.8798 - accuracy: 0.6081 - val_loss: 0.9743 - val_accuracy: 0.6014
Epoch 8/7

In [ ]:
model.save("/content/drive/MyDrive/Research Internship/val_acc_99.95")

In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
